eva.ru forum posts sentiment analysis based on different transformers models via pipeline. Due to the large number of messages and slow performance, the analysis was done in parts using code based on this notebook in a background mode. 

In [1]:
import sys
import os
import datetime
import pandas as pd
import numpy as np

In [2]:
import tensorflow as tf

In [3]:
from transformers import pipeline

In [4]:
Model='Skoltech/russian-inappropriate-messages'
column_name='inappropriate'

In [5]:
Start_Message_Id=0
End_Message_Id=10000
Part=200

In [ ]:
#Model=sys.argv[1:][0]
#column_name=sys.argv[1:][1]
#Start_Message_Id=int(sys.argv[1:][2])
#End_Message_Id=int(sys.argv[1:][3])
#Part=int(sys.argv[1:][4])

In [6]:
print('Starting Sentiment Analysis using model %s'%Model)
print('The result will be saved in %s column'%column_name)
print('Starting from message number: %s'%Start_Message_Id)
print('Ending with message number: %s'%End_Message_Id)
print('Result will be saved in %s part'%Part)

Starting Sentiment Analysis using model Skoltech/russian-inappropriate-messages
The result will be saved in inappropriate column
Starting from message number: 0
Ending with message number: 10000
Result will be saved in 200 part


In [7]:
Data='/home/kate/Projects/eva/Data'
Messages_filename='Messages.csv'
Messages_full_filename=os.path.join(Data, Messages_filename)

sa_filename='SentimentAnalysis/%s/sa_%s_%s.csv'%(column_name, column_name, Part)
sa_full_filename=os.path.join(Data, sa_filename)

In [8]:
Messages = pd.read_csv(Messages_full_filename,  index_col=False, usecols=['Message_Id','message'])

In [9]:
SubsetToAnalyze=Messages[Start_Message_Id:End_Message_Id+1:1].copy(deep=True)

In [10]:
del Messages

In [11]:
SubsetToAnalyze['message']=SubsetToAnalyze['message'].fillna(' ')

In [12]:
SubsetToAnalyze[column_name]=''

In [13]:
sentiment_analysis = pipeline('sentiment-analysis',model=Model)

Downloading:   0%|          | 0.00/870 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/542 [00:00<?, ?B/s]

In [14]:
print(datetime.datetime.now())
for index, row in SubsetToAnalyze.iterrows():
  #print(row['message'])
    if not(row['message'] != row['message']):
        try:
            sentiment = sentiment_analysis(row['message'])[0]['label']
            print('.', end = ' '),
        except:
            try:
                sentiment = sentiment_analysis(row['message'][0:1000])[0]['label']
                print('!', end = ' '),
            except:
                print('-', end = ' '),
                sentiment = 'N/A'
    else:
        print('-', end = ' '),
        sentiment = 'N/A'
    #print(sentiment)
    SubsetToAnalyze.at[index,column_name]=sentiment
    print('.', end = ' '),
print(datetime.datetime.now())      
print()
print('Processing  complete')

2021-10-29 15:55:37.370131
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

In [17]:
SubsetToAnalyze[SubsetToAnalyze['inappropriate']=='LABEL_1']

,Message_Id,message,inappropriate
123,41079715,Отличная!!! Добрая такая и вумная А пузо из ч...,LABEL_1
396,74125320,pgbooks.ru/want_to_know/new,LABEL_1
462,48399906,Пора вам пить таблетки ты что!))У вас же отста...,LABEL_1
464,48401254,"эххх Натаха, где ж такого ДОХтера то взять!",LABEL_1
510,48543362,А подскажите-сыну 2-1 не говорит ни 1 слова. В...,LABEL_1
...,...,...,...
9837,49579994,"А-а-а, жесть Моя дева зимой -- неполные 4 год...",LABEL_1
9864,49603358,папа - с пониманием относится. А вот бабушки-д...,LABEL_1
9919,49698170,"так массаж это приятно,моя ммдшная шилопопая к...",LABEL_1
9921,49698258,"а вообще как вы сами, мамы, снимаете напряжени...",LABEL_1


In [ ]:
SubsetToAnalyze.to_csv(sa_full_filename, header=True, index=False, columns=['Message_Id',column_name])  